In [2]:
import networkx as nx
import numpy as np
import time
from sklearn.decomposition import PCA

import sys
sys.path.append("..")
import utilities
from utilities import group_unfairness_score, group_unfairness_scores, recommended_nodes
import csv

In [3]:
# Modified group fairness score precomputation to include larger values of k
# id, pos_x, pos_y, inFoRM, proj_x, proj_y

In [4]:
# # networkx positions 
# def get_node_pos(G, node_features):
#     G = nx.read_edgelist(edgelist_file)
#     nodes = list(G.nodes())

#     start = time.time()
#     nodePos = nx.spring_layout(G, seed=42)
#     end = time.time()

#     for node in nodes:
#         if node not in node_features:
#             node_features[node] = {"id": node}
#         node_features[node]["pos_x"] = nodePos[node][0]
#         node_features[node]["pos_y"] = nodePos[node][1]
#     print("Spring Layout Elapsed Time: {}".format(int(end - start)))

In [5]:
# def get_pca_proj(G, embeddings, node_features):
#     start = time.time()
#     embeddings_pca = PCA(n_components=2).fit_transform(embeddings)
#     end = time.time()
#     nodes = list(G.nodes())
#     for i in range(len(nodes)):
#         node_features[nodes[i]]["proj_x"] = embeddings_pca[i][0]
#         node_features[nodes[i]]["proj_y"] = embeddings_pca[i][1]

#     print("PCA Elapsed Time: {}".format(int(end - start)))

In [6]:
# def get_inFoRM(G, embeddings, node_features):
#     adj_matrix = nx.to_numpy_array(G)
#     assert(adj_matrix.max() == 1)
#     start = time.time()
#     inFoRM_scores = utilities.unfairness_scores_normalized(embeddings, adj_matrix, G)
#     end = time.time()
#     nodes = list(G.nodes())
#     for i in range(len(inFoRM_scores)):
#         node_features[nodes[i]]["InFoRM"] = inFoRM_scores[i]
#     print("InFoRM Elapsed Time: {}".format(int(end - start)))

In [7]:
# graph_metadata = {"Facebook": {"edgelist": "../edgelists/facebook_combined.edgelist"},
#                  "LastFM": {"edgelist": "../edgelists/lastfm_asia_edges.edgelist"},
#                  "wikipedia": {"edgelist": "../edgelists/wikipedia.edgelist"},
#                   "protein-protein": {"edgelist": "../edgelists/ppi.edgelist"},
#                   "ca-HepTh": {"edgelist": "../edgelists/ca-HepTh.edgelist"},
#                   "AutonomousSystems": {"edgelist": "../edgelists/AS.edgelist"},
#                  }
graph_metadata = {"Facebook": {"edgelist": "../edgelists/facebook_combined.edgelist", 
                                "features":"../edgelists/facebook/node_genders.txt"},
                    # "Ex1": {"edgelist": "../edgelists/facebook_combined.edgelist", 
                    #             "features":"../edgelists/facebook/node_genders.txt"}          
                 }
embedding_algs = ["Node2Vec", "HOPE", "HGCN", "LaplacianEigenmap", "SDNE", "SVD"]

In [8]:
# generate csv file with ... and InFoRM fairness score
# for graph_name in graph_metadata:
#     print("\n\n" + graph_name)
#     node_features = {}

#     edgelist_file = graph_metadata[graph_name]["edgelist"]
#     G = nx.read_edgelist(edgelist_file)
#     get_node_pos(G, node_features)
    
#     for embedding_alg in embedding_algs:
#         print("\n" + embedding_alg)
#         embedding_file = "../embeddings/{}/{}/{}_{}_64_embedding.npy".format(graph_name, 
#                                                                              embedding_alg, 
#                                                                              graph_name, 
#                                                                              embedding_alg)
#         embeddings = np.load(embedding_file)
        
#         node_features_copy = node_features.copy()
#         get_inFoRM(G, embeddings, node_features_copy)
#         get_pca_proj(G, embeddings, node_features_copy)
        
#         output_file = "../embeddings/{}/{}/{}_{}_64_embedding_node_features.csv".format(graph_name, 
#                                                                                          embedding_alg, 
#                                                                                          graph_name, 
#                                                                                          embedding_alg)
#         with open(output_file, "w") as outputCSV:
#             outputCSV.write("id, pos_x, pos_y, proj_x, proj_y, InFoRM\n")
#             for node_id in node_features:
#                 outputCSV.write("{}, {}, {}, {}, {}, {}\n".format(node_features[node_id]["id"],
#                                                                   node_features[node_id]["pos_x"],
#                                                                   node_features[node_id]["pos_y"],
#                                                                   node_features[node_id]["proj_x"],
#                                                                   node_features[node_id]["proj_y"],
#                                                                   node_features[node_id]["InFoRM"]))
            

In [9]:
edgelist_file = graph_metadata["Facebook"]["edgelist"]
G = nx.read_edgelist(edgelist_file)
W = nx.to_numpy_array(G)

def dict_node_id2idx(G):
    dict_node_id2idx = {}
    for i,v in enumerate(G.nodes()):
        dict_node_id2idx[v] = i
    return dict_node_id2idx
def dict_node_idx2id(G):
    dict_node_idx2id = {}
    for i,v in enumerate(G.nodes()):
        dict_node_idx2id[i] = v
    return dict_node_idx2id

def check_symmetric(a, tol=1e-8):
    return np.all(np.abs(a-a.T) < tol)

# print(check_symmetric(W))
# print(W)
# print(not W[0,0])
# print(not W[0,1])

idx1 = dict_node_id2idx(G)['1234']
idx2 = dict_node_id2idx(G)['1111']
#print(W[idx1][idx2])

#print(G.nodes())
# for i,v in enumerate(G.nodes()):
#     if v=='1512':
#         print(i)

# for v in ['364', '501', '441', '564', '399', '393', '476', '351', '464', '578']:
#     print(dict_node_id2idx(G)[v])




embedding_file = "../embeddings/{}/{}/{}_{}_64_embedding.npy".format("Facebook", 
                                                                    "HGCN", 
                                                                    "Facebook", 
                                                                    "HGCN")
Y = np.load(embedding_file)

focal = '3984'
focal_idx = dict_node_id2idx(G)[focal]
neighbors = ['3980']
#['0', '89', '95', '147', '219', '319']
neighbors_idx = [dict_node_id2idx(G)[n] for n in neighbors]

score = 0
for n_idx in neighbors_idx:
    score += np.linalg.norm(Y[focal_idx]-Y[n_idx])**2

#print(score)

max_score = score/G.degree[focal]

print(max_score)



focal = '3812'
focal_idx = dict_node_id2idx(G)[focal]
neighbors = ['3459', '3437', '3536', '3549', '3682', '3662', '3745']
neighbors_idx = [dict_node_id2idx(G)[n] for n in neighbors]

score = 0
for n_idx in neighbors_idx:
    score += np.linalg.norm(Y[focal_idx]-Y[n_idx])**2

norm_score = (score/G.degree[focal])/max_score

print(score/G.degree[focal])

print(norm_score)


0.8927921483689403
0.7430751155719137
0.8323047160858801


In [9]:
# generate csv file with group fairness parameters and score
# nodeid, sensitive attribute (S), value (z), k, g.f. score
for graph_name in graph_metadata:
    print("\n\n" + graph_name)
    # node_features = {}

    edgelist_file = graph_metadata[graph_name]["edgelist"]
    G = nx.read_edgelist(edgelist_file)
    W = nx.to_numpy_array(G)
    
    for embedding_alg in embedding_algs:
        print("\n" + embedding_alg)
        embedding_file = "../embeddings/{}/{}/{}_{}_64_embedding.npy".format(graph_name, 
                                                                             embedding_alg, 
                                                                             graph_name, 
                                                                             embedding_alg)
        embedding = np.load(embedding_file)        

        # select output file
        output_file_gf = "../embeddings/{}/{}/{}_{}_64_embedding_recommended_nodes.csv".format(graph_name, 
                                                                                         embedding_alg, 
                                                                                         graph_name, 
                                                                                         embedding_alg)


        # create a dict nodeId->sensitive_attrs_val
        # recommended_nodes(Y,W,node_idx,k)
        # dict node_id -> row in W
        dict_node_id2idx = {}
        dict_node_idx2id = {}
        for i,v in enumerate(G.nodes()):
            dict_node_id2idx[v] = i
            dict_node_idx2id[i] = v

        with open(output_file_gf, "w") as outputCSV:
            outputCSV.write("id,k,recommended_nodes\n")

            for node_id in G.nodes():
                node_idx = dict_node_id2idx[node_id]

                for k in [25]: # define possible values for k
                    # compute recommended nodes
                    rho_u_idx = recommended_nodes(embedding,W,node_idx,k)
                    # write data to csv file
                    rho_u_id = []
                    for recnode_idx in rho_u_idx:
                        rho_u_id.append(dict_node_idx2id[recnode_idx])

                    outputCSV.write("{},{},{}\n".format(node_id,
                                            k,
                                            rho_u_id))



Facebook

Node2Vec

HOPE

HGCN

LaplacianEigenmap

SDNE

SVD
